In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import random
import os
from os.path import isfile, join
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, lupus_constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [7]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

#### The validation data

In [8]:
# test_df = pd.read_csv('../../final/data/test_set_constant.csv')
# test_df = pd.read_csv('../../lupus_trial/data/anemia/test_set_constant.csv')
val_df = pd.read_csv('../../../anemia_ml4hc/data/val_set_constant.csv')
val_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,9.496631,-1.000000,-1.000000,3.515439,375.685261,101.027779,122.174205,2.820006,1,1.276220,147.925454,74.188240,23.800042,11.545421,97.593498,28.489894,32.520362,1
1,7.092780,336.562609,-1.000000,-1.000000,115.507513,75.569193,35.449822,2.815743,0,0.870413,105.709466,118.836405,30.607322,2.113748,88.833122,21.278341,30.690491,3
2,6.554136,-1.000000,1.774495,6.145810,-1.000000,86.849700,-1.000000,2.263958,0,0.569184,139.078814,-1.000000,-1.000000,-1.000000,-1.000000,19.662409,-1.000000,6
3,12.417159,252.249921,1.701860,0.188413,515.449324,76.102948,-1.000000,4.894880,1,0.576132,4.588230,-1.000000,-1.000000,-1.000000,-1.000000,37.251478,-1.000000,3
4,7.748672,258.549935,2.980693,5.137341,363.214987,82.395181,99.002425,2.821283,1,0.885522,84.419465,33.706997,19.591641,27.675132,69.578682,23.246016,27.257252,5


In [9]:
X_val = val_df.iloc[:, 0:-1]
y_val = val_df.iloc[:, -1]

X_val, y_val = np.array(X_val), np.array(y_val)
X_val.shape, y_val.shape

((5600, 17), (5600,))

#### Validating

In [10]:
def get_steps(filename, prefix):
    try:
        return int(filename[len(prefix):][:-10])
    except Exception as e:
        print(f'Filename: {filename}')
        print(f'Exception: {e}')

In [11]:
# dqn_model = utils.load_dqn3('../../../anemia_ml4hc/models/trials/sb3/dqn3_basic_10000000_steps.zip')
# # dqn_model = utils.load_dqn3('../../final/models/dqn_noisy_6_missing_3_21300000')
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [12]:
def create_val_df(folder, X_val, y_val, prefix, suffix='zip', model_type='dqn3'):
    best_f1, best_acc, best_roc_auc = -1, -1, -1
    perf_list = []
    count = 0
        
    for item in os.listdir(folder):
        if count%10==0:
            print(count)
        count+=1
        if item.startswith(prefix):
            path = join(folder, item)
#             print(path)
            if (isfile(path)) & (path.endswith(suffix)):
                if model_type == 'dqn3':
                    model = utils.load_dqn3(path)
                else:
                    model = utils.load_dqn(path)
                model_val_df = utils.evaluate_dqn(model, X_val, y_val)
                success_rate, success_df = utils.success_rate(model_val_df)
                acc, f1, roc_auc = utils.test(model_val_df['y_actual'], model_val_df['y_pred'])
                avg_path_length, avg_return = utils.get_avg_length_reward(model_val_df)
                min_path_length, max_path_length = model_val_df.episode_length.min(), model_val_df.episode_length.max()
                min_sample_pathway =model_val_df[model_val_df.episode_length==min_path_length].iloc[0]['trajectory']
                max_sample_pathway =model_val_df[model_val_df.episode_length==max_path_length].iloc[0]['trajectory']
                perf_dict = {'steps': get_steps(item, prefix), 'acc':acc, 'f1':f1, 'roc_auc':roc_auc, 
                             'min_path_length':min_path_length, 'average_path_length':avg_path_length, 
                             'max_path_length':max_path_length, 'min_sample_pathway':min_sample_pathway, 
                             'max_sample_pathway':max_sample_pathway} 
                perf_list.append(perf_dict)
                if acc > best_acc:
                    best_acc = acc
                    model.save(f'{folder}/best_acc_model')
                if f1 > best_f1:
                    best_f1 = f1
                    model.save(f'{folder}/best_f1_model')
                if roc_auc > best_roc_auc:
                    best_roc_auc = roc_auc
                    model.save(f'{folder}/best_roc_auc_model')

#     print('creating dataframe object')
    val_df = pd.DataFrame.from_dict(perf_list) 
    val_df = val_df.sort_values(by=['steps'])
    val_df = val_df.reset_index(drop=True)
#     print('saving validation results')
    val_df.to_csv(f'{folder}/validation_results.csv', index=False)
    return val_df

In [13]:
folder = '../../../anemia_ml4hc/models/sb/dqn/logs'
prefix = 'dqn_basic_'
val_df = create_val_df(folder, X_val, y_val, prefix, suffix='zip', model_type='dqn')
val_df.head()

0
Using just stable baselines (not 3)
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.cast instead.


Invalid MIT-MAGIC-COOKIE-1 key

Testing done.....
Using just stable baselines (not 3)
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Testing done.....
Using just stable baselines (not 3)
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Testing done.....
Using just stable baselines (not 3)
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Testing done.....
Using just stable baselines (not 3)
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Testing done.....
Using just stable baselines (not 3)
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Testing done.....
Using just stable baselines (not 3)
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Testing done.....
Using just stable baselines (not 3)
Load

Testing done.....
Using just stable baselines (not 3)
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Testing done.....
Using just stable baselines (not 3)
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Testing done.....
Using just stable baselines (not 3)
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Testing done.....
Using just stable baselines (not 3)
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Testing done.....
Using just stable baselines (not 3)
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Testing done.....
Using just stable baselines (not 3)
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Testing done.....
60
Using just stable baselines (not 3)
L

,steps,acc,f1,roc_auc,min_path_length,average_path_length,max_path_length,min_sample_pathway,max_sample_pathway
0,100000,9.607143,2.191268,50.0,2.0,2.697321,6.0,"[tibc, Inconclusive diagnosis]","[tibc, mcv, ferritin, serum_iron, copper, Inco..."
1,200000,9.607143,2.191268,50.0,3.0,3.000000,3.0,"[hemoglobin, glucose, Inconclusive diagnosis]","[hemoglobin, glucose, Inconclusive diagnosis]"
2,300000,9.607143,2.191268,50.0,2.0,3.169286,5.0,"[segmented_neutrophils, Inconclusive diagnosis]","[segmented_neutrophils, folate, serum_iron, cr..."
3,400000,9.607143,2.191268,50.0,2.0,2.726964,4.0,"[glucose, Inconclusive diagnosis]","[glucose, tibc, creatinine, Inconclusive diagn..."
4,500000,9.607143,2.191268,50.0,2.0,2.833750,7.0,"[cholestrol, Inconclusive diagnosis]","[cholestrol, tsat, glucose, tibc, serum_iron, ..."
